In [5]:
from __future__ import print_function

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2.credentials import Credentials
from azure.storage.blob import BlobServiceClient

import pandas as pd
import os.path
import json
pd.set_option('display.max_rows', 500)

In [6]:
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']
root_dir =  os.path.abspath(os.path.join(os.getcwd(), "../../.."))
tmp_dir = root_dir + "/.tmp"
cred_dir = root_dir + "/.cred"
data_dir = root_dir + "/data"
site_summary_filename = data_dir + "hourly-site-summary.csv"

## Establish Azure Connection - waiting for pw

## Generate Hourly Dataset

In [8]:
## TEMP ##
# Set paths
wd = '/Users/jetcalz07/Desktop/MIDS/W210_Capstone/co2-flux-hourly-gpp-modeling/'
data = wd + 'data/datasets/'
hh_data = data + 'half_hourly/'
code = wd + 'code/src/'

In [10]:
# Loop through directory, loading HH site data
site = os.listdir(hh_data)[1]
site_hh = pd.read_csv(hh_data + site)
site_hh.head()

,TIMESTAMP_START,TIMESTAMP_END,TA_F,TA_F_QC,TA_ERA,SW_IN_POT,SW_IN_F,SW_IN_F_QC,SW_IN_ERA,LW_IN_F,...,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen
0,200201150000,200201150030,-8.275,0,-14.030,0.0,0.0,0,0.0,238.465,...,-0.006405,0.9252,0.9123,0.8571,0.9113,0.5601,0.1609,0.0708,GRA,Polar
1,200201150030,200201150100,-8.926,0,-14.124,0.0,0.0,0,0.0,238.465,...,-0.006405,0.9252,0.9123,0.8571,0.9113,0.5601,0.1609,0.0708,GRA,Polar
2,200201150100,200201150130,-9.717,0,-14.218,0.0,0.0,0,0.0,238.465,...,-0.006405,0.9252,0.9123,0.8571,0.9113,0.5601,0.1609,0.0708,GRA,Polar
3,200201150130,200201150200,-10.092,0,-14.311,0.0,0.0,0,0.0,238.465,...,-0.006405,0.9252,0.9123,0.8571,0.9113,0.5601,0.1609,0.0708,GRA,Polar
4,200201150200,200201150230,-10.173,0,-14.405,0.0,0.0,0,0.0,228.640,...,-0.006405,0.9252,0.9123,0.8571,0.9113,0.5601,0.1609,0.0708,GRA,Polar


In [19]:
# Subset records to only those with TIME_START ending in 00
site_hh['TIMESTAMP_START'] = site_hh['TIMESTAMP_START'].astype(str)
site_hr = site_hh.loc[site_hh['TIMESTAMP_START'].str[-2:]=='00', ]